In [292]:
import pandas as pd 
import solid 
from datetime import datetime, timedelta
from urllib import urlencode
from math import * 
from subprocess import call


In [3]:
data = pd.read_csv('https://data.cityofchicago.org/api/views/9m5u-3f29/rows.csv?accessType=DOWNLOAD')

In [26]:
data.describe()

,Average Days to Complete Graffiti Removal,Total Completed Requests,Median Days to Complete Graffiti Removal Request,Target Response Time (Days)
count,28.000000,28.000000,27.000000,2
mean,7.456429,2099.785714,2.962963,10
std,2.088088,340.938830,1.192331,0
min,3.100000,1430.000000,1.000000,10
25%,6.047500,1927.500000,2.000000,10
50%,7.325000,2160.000000,3.000000,10
75%,9.175000,2309.000000,4.000000,10
max,11.570000,2706.000000,5.000000,10


In [14]:
polygon =solid.polygon(([1,2],[2,3],[1,3],[2,2]))

In [20]:
from solid.utils import *
extrude_along_path

<function solid.utils.extrude_along_path>

In [21]:
def path():
    return [Point3(0,0,0), Point3(0,0,1)]

In [24]:
shape =extrude_along_path(shape_pts=[Point3(1,2,0),Point3(2,3,0),Point3(1,3,0),Point3(2,2,0)], path_pts=path())



polyhedron(points = [[0.0000000000, 0.0000000000, 0.0000000000], [0.0000000000, 0.0000000000, 0.0000000000], [0.0000000000, 0.0000000000, 0.0000000000], [0.0000000000, 0.0000000000, 0.0000000000], [0.0000000000, 0.0000000000, 1.0000000000], [0.0000000000, 0.0000000000, 1.0000000000], [0.0000000000, 0.0000000000, 1.0000000000], [0.0000000000, 0.0000000000, 1.0000000000]], triangles = [[0, 4, 1], [1, 4, 5], [1, 5, 2], [2, 5, 6], [2, 6, 3], [3, 6, 7], [0, 3, 7], [0, 7, 4], [0, 1, 2], [0, 2, 3], [4, 6, 5], [4, 7, 6]]);


In [16]:
from solid import *
d = difference()(
    cube(10),
    sphere(15)
)
print(scad_render(d))



difference() {
	cube(size = 10);
	sphere(r = 15);
}


## Get data

In [99]:
a = []
for i in range(1,13):
    monthly = pd.read_csv('https://www.ncdc.noaa.gov/cag/time-series/global/globe/land_ocean/1/{0}/1880-2015.csv'.format(i), header=3)
    monthly['month'] = i
    a.append(monthly)
total = pd.concat(a)

In [240]:
def angle(month, total_segments):
    return (month/float(total_segments))*2*pi

In [284]:
def radius(data,min_val, max_val,max=20,min=1):
    return (data - min_val)*(max-min)/(max_val-min_val) + min

In [288]:
def segment_path( values, segments, height= 20, max_r=10, min_r=1):
    shape = [[radius(d,min_r,max_r)*sin(angle(segment-1,len(segments))), radius(d,min_r,max_r)*cos(angle(segment-1,len(segments)))] for segment,d in zip(segments, values)]
    return linear_extrude(height=height)(polygon(points=shape))

In [289]:
def make_sculpture(data, vertical_col, segment_col, value_col,segment_height=2):
    segments = []
    max_r = data[value_col].max()
    min_r = data[value_col].min()
    for offset, vCol  in enumerate(range(data[vertical_col].min(), data[vertical_col].max())):
        section_data = data[data[vertical_col]==vCol].sort(columns=segment_col, ascending=True)
        segments.append(up(segment_height*offset)(segment_path(section_data[value_col], section_data[segment_col],height=segment_height, max_r= max_r, min_r=min_r)))
    final = union()(segments)
    return final

In [254]:
final = make_sculpture(total, "Year", 'month', 'Value')

In [255]:
with open('sculpture.scad', 'w') as f:
    f.write(scad_render(final).replace('triangles','faces'))

In [256]:
total['Year'].max()

2015

# Chicago Crime Data 

In [ ]:
query= {'q':'''
    select week,year,primary_type, count from crimes_2001_to_present_summary
''', 'format':'csv'}

crimes = pd.read_csv('https://stuartlynn.cartodb.com/api/v2/sql?'+urlencode(query))

In [276]:
def tfunc(x):
    return datetime(x['year'],1,1) + timedelta(weeks=x['week']-1)

In [293]:
for crime in crimes['primary_type'].unique():
    print 'doing :', crime
    data = crimes[crimes['primary_type']==crime]
    data.index = data.apply(lambda x : tfunc(x), axis=1) 
    data.resample('1w').bfill(0)
    data['year'] = data.index.year
    data['week'] = data.index.week
    sculpt = make_sculpture(data, 'year', 'week','count',segment_height=4)
    with open('chicago_crimes/{0}.scad'.format(crime), 'w') as f:
        f.write(scad_render(sculpt).replace('triangles','faces'))
    call(['/Applications/OpenSCAD.app/Contents/MacOS/OpenSCAD',
          '-o', 
          'chicago_crimes/{0}.stl'.format(crime), 
          'chicago_crimes/{0}.scad'.format(crime)])

doing : ARSON


/Users/stuartlynn/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/stuartlynn/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


doing : ASSAULT
doing : BATTERY
doing : BURGLARY
doing : CRIMINAL DAMAGE
doing : CRIMINAL TRESPASS
doing : CRIM SEXUAL ASSAULT
doing : DECEPTIVE PRACTICE
doing : GAMBLING
doing : HOMICIDE
doing : INTERFERENCE WITH PUBLIC OFFICER
doing : INTIMIDATION
doing : KIDNAPPING
doing : LIQUOR LAW VIOLATION
doing : MOTOR VEHICLE THEFT
doing : NARCOTICS
doing : OFFENSE INVOLVING CHILDREN
doing : OTHER OFFENSE
doing : PROSTITUTION
doing : PUBLIC PEACE VIOLATION
doing : RITUALISM
doing : ROBBERY
doing : SEX OFFENSE
doing : STALKING
doing : THEFT
doing : WEAPONS VIOLATION
doing : OBSCENITY
doing : OTHER NARCOTIC VIOLATION
doing : PUBLIC INDECENCY
doing : HUMAN TRAFFICKING
doing : NON-CRIMINAL (SUBJECT SPECIFIED)
doing : DOMESTIC VIOLENCE
doing : NON - CRIMINAL
doing : CONCEALED CARRY LICENSE VIOLATION
doing : NON-CRIMINAL
